In [ ]:
#| default_exp process

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import *

In [ ]:
#| export
import configparser
import os
from fastcore.basics import patch
from os.path import expanduser
from astroquery.astrometry_net import AstrometryNet
from astroquery.exceptions import TimeoutError as ASTTimeoutError
from osob.core import Telescope
from osob.solver import Solver
from tqdm.auto import tqdm

In [ ]:
#| local
config = configparser.ConfigParser()
config.read(expanduser('~/.config/telescope.ini'))

['/home/jochym/.config/telescope.ini']

In [ ]:
#| local
OSO=Telescope(config['telescope.org']['user'], 
              config['telescope.org']['password'])

In [ ]:
#| local
def get_object_obs(obj):
    reqlst=OSO.get_user_requests(sort='completion')
    print(f'Number of users requests: {len(reqlst)}')

    complete = [rq for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True) 
                        if Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]=='Complete']
    print('Completed:', len(complete))
    objjobs = ((int(OSO.get_request(int(j['id']))['jid'][1:]), int(j['id'])) 
               for j in complete if j['objectname']==obj)
    return objjobs

In [ ]:
#| local
reqlst=OSO.get_user_requests(sort='completion')
complete = [rq for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True) 
                        if Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]=='Complete']

In [ ]:
#| local

# Get all observations of VS
VS = 'BI Her'
jlist = list(get_object_obs(VS))
print(f'Completed observations of {VS}: {len(jlist)}')

for jid, rid in tqdm(jlist):
    # print(f"R{rid}:J{jid}")
    try :
        data = OSO.get_obs(OSO.get_job(jid), cube=True, verbose=False)
        if data :
            fp = f'VS/{"_".join(VS.split())}'
            os.makedirs(fp, exist_ok=True)
            fn = f'{jid}'
            try :
                # print(os.path.abspath(data.name), f'{fp}/{fn}.fits')
                os.symlink(os.path.abspath(data.name), f'{fp}/{fn}.fits')
            except FileExistsError:
                pass
            # with open(f'/home/jochym/Astro/VS/{"_".join(VS.split())}/{jid}.fits', 'wb') as ff:
            #     ff.write(data.read())
        else :
            print(f'Download of J{jid} failed')
    except TimeoutError:
        print(f'Download of J{jid} failed')
        continue

Number of users requests: 1439
Completed: 1414
Completed observations of BI Her: 143


  0%|          | 0/143 [00:00<?, ?it/s]

In [ ]:
#| local

# get all completed jobs
for rq in (pbar := tqdm(complete)):
    name = rq['objectname']
    rid = int(rq['id'])
    pbar.set_postfix_str(name)
    try:
        jid = int(OSO.get_request(rid)['jid'][1:])
        data = OSO.get_obs(OSO.get_job(jid), cube=True, verbose=False)
        if data :
            fp = f'VS/{"_".join(name.split())}'
            os.makedirs(fp, exist_ok=True)
            fn = f'{jid}'
            try :
                os.symlink(os.path.abspath(data.name), f'{fp}/{fn}.fits')
            except FileExistsError:
                pass
        else :
            print(f'Download of J{jid} failed')
    except TimeoutError :
        print(f'Download of J{jid} failed, timeout')
        pass

  0%|          | 0/1414 [00:00<?, ?it/s]

Download of J321992 failed, timeout
Download of J321989 failed, timeout
Download of J294696 failed, timeout
Download of J293658 failed, timeout
Download of J293657 failed, timeout
Download of J293656 failed, timeout
Download of J293655 failed, timeout
Download of J293654 failed, timeout
Download of J293653 failed, timeout
Download of J293652 failed, timeout
Download of J293651 failed, timeout
